# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/naga/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/naga/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [5]:
!mkdir data

mkdir: data: File exists


In [6]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31493    0 31493    0     0   119k      0 --:--:-- --:--:-- --:--:--  119k


In [7]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70519    0 70519    0     0   433k      0 --:--:-- --:--:-- --:--:--  435k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [14]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [9]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/naga/new/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/naga/new/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/naga/new/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [10]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [11]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [12]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/20 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 11, relationships: 42)

We can save and load our knowledge graphs as follows.

In [15]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 11, relationships: 42)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [17]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [18]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is Hugging Face known for in the context ...,[My blog in 2023 Here’s the sequel to this pos...,Hugging Face is mentioned as having a communit...,single_hop_specifc_query_synthesizer
1,How does OpenAI relate to the challenges and e...,[openly licensed ones are still the world’s mo...,The context mentions that almost every major L...,single_hop_specifc_query_synthesizer
2,Artificial Intelligence is what?,[Simon Willison’s Weblog Subscribe Stuff we fi...,"Artificial Intelligence, as discussed in the c...",single_hop_specifc_query_synthesizer
3,What is OpenAI's role in the development of la...,"[of what LLMs are, how they work and how they ...",The context discusses OpenAI as a significant ...,single_hop_specifc_query_synthesizer
4,How has Google contributed to advancements in ...,[Things we learned about LLMs in 2024 31st Dec...,Google contributed to advancements in large la...,single_hop_specifc_query_synthesizer
5,How do recent developments in inference scalin...,"[<1-hop>\n\non inference. The sequel to o1, o3...","Recent developments in inference scaling, exem...",multi_hop_abstract_query_synthesizer
6,How do the emerging challenges of AI agents re...,"[<1-hop>\n\nof what LLMs are, how they work an...",The context highlights the emergence and chall...,multi_hop_abstract_query_synthesizer
7,How do recent developments and experimentation...,[<1-hop>\n\nthis. LLMs need better criticism A...,The context highlights that while LLMs face cr...,multi_hop_abstract_query_synthesizer
8,Llama 3.2 is like super good but also how it l...,[<1-hop>\n\npunch massively above their weight...,"The context explains that Llama 3.2, specifica...",multi_hop_specific_query_synthesizer
9,Considering the advancements in GPT-4 as discu...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,Simon Willison’s weblog highlights that 2023 w...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [19]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/22 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [21]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Replit is what?,[My blog in 2023 Here’s the sequel to this pos...,Replit is mentioned as one of the organization...,single_hop_specifc_query_synthesizer
1,"As an AI researcher and developer, how does th...",[openly licensed ones are still the world’s mo...,The context mentions that almost every major L...,single_hop_specifc_query_synthesizer
2,what are LLMs and why they are important in AI?,[Simon Willison’s Weblog Subscribe Stuff we fi...,Large Language Models (LLMs) are a recent brea...,single_hop_specifc_query_synthesizer
3,What is Llama in the context of large language...,"[of what LLMs are, how they work and how they ...",Llama is mentioned among the tag cloud words r...,single_hop_specifc_query_synthesizer
4,How do the recent advancements in voice and li...,[<1-hop>\n\na similar way to ChatGPT now. Amaz...,The recent advancements in voice and live came...,multi_hop_abstract_query_synthesizer
5,How do recent developments in API accessibilit...,[<1-hop>\n\na similar way to ChatGPT now. Amaz...,Recent advancements like OpenAI's new WebRTC A...,multi_hop_abstract_query_synthesizer
6,how can we use large language models for real ...,[<1-hop>\n\nThe year of slop Synthetic trainin...,"Based on the recent context, large language mo...",multi_hop_abstract_query_synthesizer
7,how model training and efficiency improvements...,[<1-hop>\n\nThe year of slop Synthetic trainin...,"The context shows that over 2024, there have b...",multi_hop_abstract_query_synthesizer
8,How do the developments in Llama 3.1 and DeepS...,"[<1-hop>\n\nof what LLMs are, how they work an...","The context shows that DeepSeek v3, a large 68...",multi_hop_specific_query_synthesizer
9,Whay is Claude 3 and how it compare to Claude?,[<1-hop>\n\nPhi series of models has consisten...,Claude 3 is a model from the Claude series of ...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [25]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [26]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [27]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [29]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [30]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

/Users/naga/new/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:758: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is empty, alternative syntax for `is_empty: \&quot;field_name\&quot;`",
/Users/naga/new/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:762: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is null, alternative syntax for `is_null: \&quot;field_name\&quot;`",


In [31]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [32]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [33]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [34]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [35]:
rag_chain.invoke({"question" : "What are Agents?"})

'Based on the provided context, "agents" is an infuriatingly vague term without a single, clear, and widely understood meaning. Generally, the term "agents" tends to refer to AI systems that can go away and act on your behalf. There are two main interpretations: one is the "travel agent" model, where agents act autonomously to perform tasks for a user, and the other involves large language models (LLMs) given access to tools which they can use in a loop to solve problems. However, despite much discussion and many prototypes, true AI agents—systems that reliably and autonomously act on behalf of users—have not yet been realized or widely deployed in production. The concept is complicated by challenges such as gullibility of LLMs and the lack of a clear definition of autonomy. Some believe fully functional agents may depend on achieving Artificial General Intelligence (AGI). In summary, agents are envisioned as autonomous AI systems acting on users\' behalf, but the term remains vague an

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [36]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [37]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dope_or_nope_evaluator`:

## LangSmith Evaluation

In [38]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'loyal-payment-68' at:
https://smith.langchain.com/o/163edaa3-63be-4fba-8c52-9a879cd2bd4b/datasets/b4ecaca5-a595-4604-a37d-8cecfde5233f/compare?selectedSessions=f05b28d6-e024-46f2-84a5-718f58d8a76e




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"How does the development of Claude, as discuss...","Based on the context, Claude, particularly wit...",None,"The context highlights that Claude, along with...",1,1,0,6.372785,6db78a4b-b35e-4f51-8f82-a551eb530303,3545e3fb-0883-4936-9b93-1810a648514a
1,How do recent advancements in large language m...,Recent advancements in large language models t...,None,"Recent advancements, exemplified by DeepSeek v...",1,1,0,5.615689,cf16241f-b4c9-43dc-b8d3-d245a32e3a9c,44d1122f-dd7c-4ebf-84d2-7742487de877
2,Whay is Claude 3 and how it compare to Claude?,Claude 3 refers to a series of models released...,None,Claude 3 is a model from the Claude series of ...,1,0,0,3.738666,5ff41bbd-0eb7-441c-a61e-96de350dfe00,2a86cb36-a8fb-4aa5-bdba-1fc6ba75a193
3,How do the developments in Llama 3.1 and DeepS...,The developments in Llama 3.1 and DeepSeek v3 ...,None,"The context shows that DeepSeek v3, a large 68...",1,1,0,8.450389,a459ffdb-441b-438b-af9e-c262ad363fbc,5f6f38aa-c68b-4b89-9be9-607e7ee954ff
4,how model training and efficiency improvements...,Model training and efficiency improvements hel...,None,"The context shows that over 2024, there have b...",1,1,0,5.142479,92ef86ac-a782-4001-a338-af44889016c0,5df52a2f-dc26-43da-a974-754172495225
5,how can we use large language models for real ...,"Based on the provided context, large language ...",None,"Based on the recent context, large language mo...",1,1,0,8.544403,9a6fa01b-3501-4441-8907-cf6a7227986a,32f69545-f78e-439d-80cc-e3e7eea2e715
6,How do recent developments in API accessibilit...,Recent developments in API accessibility and a...,None,Recent advancements like OpenAI's new WebRTC A...,1,1,0,6.639437,a99638e2-fe7c-45db-8962-5fac93912627,b69f8bb5-5897-4f68-b4d5-79ce756ffa80
7,How do the recent advancements in voice and li...,The recent advancements in voice and live came...,None,The recent advancements in voice and live came...,1,1,0,3.505557,980902c3-a234-445d-88dd-da4a18d00f80,e1467c0b-1a8a-4a8e-b586-2295128256db
8,What is Llama in the context of large language...,Llama is a large language model released by Me...,None,Llama is mentioned among the tag cloud words r...,1,1,0,1.818250,8d11c908-c20f-4799-bf75-6cdb6ca48d6c,d31724c5-ef6b-4c87-90de-9e5125ec1e73
9,what are LLMs and why they are important in AI?,"Based on the provided context:\n\nLLMs, or Lar...",None,Large Language Models (LLMs) are a recent brea...,1,1,0,3.260459,485a48eb-04b9-4b0a-a0f2-b6de2d8d2c5f,420ff508-eb4f-45d2-a0e7-bf1fc366f66a


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [39]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [40]:
rag_documents = docs

In [41]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

In [42]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [43]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [44]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [45]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [46]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Alright, listen up! Agents are kinda like these AI homies that are supposed to roll out and do stuff *on your behalf*—think travel agents but digital style. But here’s the kicker: the term “agents” is *mad* vague, with no solid, universal definition. Some folks see them as AI that acts on your behalf, others as language models hooked up to tools, running loops to solve problems. And people throw around “autonomy” like it’s a magic word, but no one really nails down what that means either.\n\nThe real tea? These agents are still more “coming soon” vibes than actual deployed legends. A massive roadblock is gullibility—LLMs just believe anything you feed ’em, making trust a shaky game. Without AGI-level smarts, sorting truth from fiction and handling prompt injection nonsense is a beast we haven’t tamed yet. So yeah, agents are this futuristic, somewhat mysterious concept of AI that acts independently for you…but the jury’s still out on how real and reliable they are. Keep it chill, they

Finally, we can evaluate the new chain on the same test set!

In [47]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'damp-hope-52' at:
https://smith.langchain.com/o/163edaa3-63be-4fba-8c52-9a879cd2bd4b/datasets/b4ecaca5-a595-4604-a37d-8cecfde5233f/compare?selectedSessions=102ea5ff-800a-4a49-8c0b-143f26e7e231




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"How does the development of Claude, as discuss...","Yo, here’s the lowdown straight from the conte...",None,"The context highlights that Claude, along with...",1,1,1,6.170217,6db78a4b-b35e-4f51-8f82-a551eb530303,c9527bcf-c969-4074-8b05-8af9e931ddaa
1,How do recent advancements in large language m...,"Yo, here’s the lowdown straight from the conte...",None,"Recent advancements, exemplified by DeepSeek v...",1,1,1,4.808050,cf16241f-b4c9-43dc-b8d3-d245a32e3a9c,b7f935a3-1a04-48e3-9220-4481c8336569
2,Whay is Claude 3 and how it compare to Claude?,"Alright, here’s the lowdown on Claude 3 and ho...",None,Claude 3 is a model from the Claude series of ...,1,0,1,3.577538,5ff41bbd-0eb7-441c-a61e-96de350dfe00,7e39492c-61f4-4d65-b502-5f75cdf9f0dd
3,How do the developments in Llama 3.1 and DeepS...,"Yo, check this out — Llama 3.1 and DeepSeek v3...",None,"The context shows that DeepSeek v3, a large 68...",1,1,1,4.654805,a459ffdb-441b-438b-af9e-c262ad363fbc,497e696b-bc4b-4a37-a85d-7d3cbd571d7b
4,how model training and efficiency improvements...,"Alright, check it out—model training and effic...",None,"The context shows that over 2024, there have b...",1,1,1,3.784412,92ef86ac-a782-4001-a338-af44889016c0,3a36c197-ccc8-4144-bfba-a66b31e64caa
5,how can we use large language models for real ...,"Yo, here’s the lowdown straight from the conte...",None,"Based on the recent context, large language mo...",1,1,1,9.104633,9a6fa01b-3501-4441-8907-cf6a7227986a,5c7236a7-05fc-4d51-a242-4cb2618f0a63
6,How do recent developments in API accessibilit...,"Alright, check this out — the game’s changed i...",None,Recent advancements like OpenAI's new WebRTC A...,1,1,1,4.477691,a99638e2-fe7c-45db-8962-5fac93912627,39663edb-e305-469c-9f73-73aaa929cf0d
7,How do the recent advancements in voice and li...,"Yo, check this out — the dope advancements in ...",None,The recent advancements in voice and live came...,1,1,1,3.680232,980902c3-a234-445d-88dd-da4a18d00f80,320b0e7a-db28-47b3-896a-66442bea03af
8,What is Llama in the context of large language...,"Alright, here’s the skinny on Llama in the AI ...",None,Llama is mentioned among the tag cloud words r...,1,1,1,6.103636,8d11c908-c20f-4799-bf75-6cdb6ca48d6c,cf945573-bfc9-4d23-9454-a8d78dbc6cd3
9,what are LLMs and why they are important in AI?,"Alright, here’s the lowdown on LLMs—those big ...",None,Large Language Models (LLMs) are a recent brea...,1,1,1,6.338738,485a48eb-04b9-4b0a-a0f2-b6de2d8d2c5f,d638d1ce-6e73-4541-9d5f-e3cd0588ddbe


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.